In [151]:
###########################
# Pre-processing the data #
###########################


import pandas as pd
import numpy as np
###########################
# Pre-processing the data #
###########################

from keras.optimizers import SGD
from sklearn.preprocessing import scale
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential, Model
from keras.layers import Dense,BatchNormalization
from keras.layers import LSTM, Input
from keras.callbacks import Callback
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error,r2_score

debug=False
files = ['miller_idle', 'sam_wild', 'layton_idle', 'miller_wild', 'val_idle', 'layton_wild', 'sam_idle', 'val_wild']  #filename in the folder
STANDARD_SCALER = True
PROGRAMMATIC = True

#params for the net:
layers   = [30, 10]
dropouts = [.2, .2]
activations = ['relu','relu']#*4#['relu','relu','relu','relu']
recurrent_dropouts = [.2,.2]
epochs = 30 
batch_size = 10
unit_forget_bias = True
#Initializers: he_normal, glorot_normal
kernel_initializer = 'lecun_uniform'
recurrent_initializer = 'lecun_uniform'
go_backwards = True
LSTM_FLAG = True

'''
hyper params: 
    activation functions
    number of layers
    number of neurons per layer 
    optimizer
    learning rate
    epochs
    batch size 
    unit_forget_bias
'''

'\nhyper params: \n    activation functions\n    number of layers\n    number of neurons per layer \n    optimizer\n    learning rate\n    epochs\n    batch size \n    unit_forget_bias\n'

In [142]:
class R2Callback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        X_test, y_test = self.test_data
        pred = self.model.predict(X_test)
        r2 = r2_score(pred,y_test)
        print('r2 score: {},\n'.format(r2))

In [143]:
#def makeLSTM(layer_list,layers,activation,dropouts):
def makeLSTM(visible,layers,activation,dropouts):
    assert(len(layers)==len(dropouts) and 
           len(layers)==len(activations) and 
           len(layers)==len(recurrent_dropouts) )
    
    cell = LSTM if LSTM_FLAG else GRU()
    '''
    Mimicing the following:
    visible = ...
    extract = LSTM(30, activation="relu",dropout=0.5,return_sequences=True)(visible)
    extract = LSTM(20, activation="relu",dropout=0.5,return_sequences=True)(extract)
    extract = LSTM(10, activation="relu",dropout=0.5,return_sequences=True)(extract)
    extract = LSTM(5, activation="relu",dropout=0.5,return_sequences=True)(extract)
    '''
    info = zip(layers,dropouts,activations)
    for i in range(len(layers)): #layer,dropout,activation in info:
        if debug: 
            print("layer: {},dropout: {}, activation: {}".format(layers[i],dropouts[i],activations[i]))
        #visible = LSTM(layers[i],
        visible = cell(layers[i],
                       activation = activations[i], 
                       dropout = dropouts[i],
                       recurrent_dropout = recurrent_dropouts[i],
                       return_sequences=True, 
                       unit_forget_bias=unit_forget_bias,
                       kernel_initializer = kernel_initializer,
                       recurrent_initializer = recurrent_initializer)(visible)
        #layer_list += LSTM(layers[i],activation = activations[i], dropout = dropouts[i], return_sequences=True)(layer_list[-1])
    #return layer_list[-1]
    return visible

In [144]:
#scaler should be a scaling object: either StandardScaler or MinMaxScaler
def preprocessing(filename,scaler):
    
    d = pd.read_csv(filename)   #read the CSV file
    if debug:
        print("Original {}".format(d))
        print (d)
    d_drop = d.drop("Sensor type", axis=1)   #drop the first column -- sensor type
    if debug:
        print ("Dropped Sensor Type {}".format(d_drop))
    d_matrix = d_drop.as_matrix()   # Using pandas again to convert the revised table into a matrix

    n_array = np.array(d_matrix) # now we use numpy to convert the pandas matrix into a Numpy array

    #delete the last row if the revised row number is odd, otherwise cut last too rows
    n_array = n_array[:-1] if len(n_array) % 2 == 1 else n_array[:-2] 
    if debug:
        print ("Delete odd rows \n{}".format(n_array[:16,:]))

    #Concatenate the EMG and Accerlaration rows for (nearly) the same time moments
    n_conc = np.concatenate((n_array[::2], n_array[1::2]), axis = 1)    
    if debug:
        print ("Concatenated array \n{}".format(n_conc[:16,:]))

    n_del = np.delete(n_conc, [1, 2], axis = 1) #delete the second and third columns of each list in the matrix
    if debug:
        print ("Delete Zero rows \n{}".format(n_del[:16,:]))
        print("n_del.shape: {}".format(n_del.shape))
        
    targets = targets = n_del[1:, 2:5] #target for data t is [x,y,z] of t+1
    data = n_del[:-1] # no target for last example, so drop it.
    
    if debug: 
        print("data.shape: {}".format(data.shape))
    #Normalize the data using the supplied scaler
    data = scaler.fit(data).transform(data)
    targets = scaler.fit(targets).transform(targets)
        
    return data, targets

def save_array(array, filename):
    np.savetxt(filename + 'processed', array, delimiter=',', fmt = '%f')  #save the output into a file


In [145]:
num_attributes = 6
num_targets = 3

scaler = StandardScaler() if STANDARD_SCALER else MinMaxScaler()

data = np.empty((0,num_attributes)) #initializing empty array with proper shape
targets = np.empty((0,num_targets)) #empty array for targets

#collecting all the data into one set
for f in files:
    processed_data, processed_targets = preprocessing(f,scaler)
    data = np.append(data, processed_data, axis=0)
    targets = np.append(targets, processed_targets, axis=0)
    save_array(processed_data, f)
    

#data = scale(data)

if debug : print("data[0]: {}".format(data[0]))
temp = list()
for i in range(1,targets.shape[0]):
    #temp = targets[i]-targets[i-1]
    temp.append(targets[i]-targets[i-1])
input_data = list()
for i in range(0,data.shape[0]):
    input_data.append([data[i]])
    
input_data = np.array(data)
acceleration_targets = np.array(temp)
data = data.reshape((data.shape[0],1,data.shape[1]))[1:]
if debug : print("data[0:10]: {}".format(data[0:10]))


In [146]:
from sklearn.model_selection import train_test_split

X_train, y_train, X_test, y_test = data[0:40000],acceleration_targets[0:40000],data[40000:],acceleration_targets[40000:]

In [ ]:
#https://machinelearningmastery.com/keras-functional-api-deep-learning/
visible = Input(shape=(1,6))
# feature extraction



if PROGRAMMATIC: 
# classification output
    #layer = makeLSTM([visible],layers,activations,dropouts)
    layer = makeLSTM(visible,layers,activations,dropouts)
    class11 = LSTM(3,activation=None)(layer)
    output1 = Dense(3, activation='linear')(class11)
    model = Model(inputs=visible, outputs=output1)

else:
    extract = LSTM(30, activation="relu",dropout=0.5,return_sequences=True)(visible)
    extract = LSTM(20, activation="relu",dropout=0.5,return_sequences=True)(extract)
    extract = LSTM(10, activation="relu",dropout=0.5,return_sequences=True)(extract)
    extract = LSTM(5, activation="relu",dropout=0.5,return_sequences=True)(extract)
    class11 = LSTM(3,activation=None)(extract)
    output1 = Dense(3, activation='linear')(class11)
    model = Model(inputs=visible, outputs=output1)

# summarize layers
print(model.summary())
#sgd= SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='mean_squared_error',optimizer="adam",metrics=["mae"])
model.fit(X_train,y_train,verbose=1,batch_size=batch_size,epochs=epochs,validation_data=(X_test,y_test),callbacks=[R2Callback((X_test,y_test))])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 1, 6)              0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 1, 30)             4440      
_________________________________________________________________
lstm_24 (LSTM)               (None, 1, 10)             1640      
_________________________________________________________________
lstm_25 (LSTM)               (None, 3)                 168       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 12        
Total params: 6,260
Trainable params: 6,260
Non-trainable params: 0
_________________________________________________________________
None
Train on 40000 samples, validate on 34121 samples
Epoch 1/30
40000/40000 [==============================] - 34s 857us/step - loss: 0.17

In [102]:
print('data:')
print(X_test[:10])
print('\npredictions:')
print(model.predict(X_test)[:10])
print('\ntargets:')
print(y_test[:10])

data:
[[[ 0.03710197  1.62231395 -0.10282867  1.92876797 -0.51208322
    1.62231101]]

 [[-0.00751908  1.62254565 -0.1113402   1.52475057 -0.4092953
    1.62254271]]

 [[-0.03870005  1.62275628 -0.09431714  1.39036525 -0.29127953
    1.62275335]]

 [[-0.06934342  1.62298798 -0.07729409  1.08713684 -0.31792825
    1.62298505]]

 [[-0.09514836  1.62324074 -0.1113402   0.95275152 -0.33315609
    1.62323781]]

 [[-0.07579466  1.62345138 -0.11985173  1.01994418 -0.37883962
    1.62346951]]

 [[ 0.10430234  1.62368307 -0.13687479  1.35590748 -0.52731106
    1.62368014]]

 [[ 0.11344159  1.62389371 -0.1709209   1.42396158 -0.4663997
    1.62389078]]

 [[ 0.06183171  1.62412541 -0.15389784  1.35590748 -0.37883962
    1.62412247]]

 [[ 0.01774826  1.62433604 -0.1709209   1.22152216 -0.24559601
    1.62433311]]]

predictions:
[[ 0.00028317 -0.02860187  0.06666152]
 [ 0.00766453 -0.01773301  0.06156803]
 [ 0.01276261 -0.01676627  0.05601323]
 [ 0.01365463 -0.00872439  0.06249333]
 [ 0.01394914 -0

In [103]:
print('data:')
print(X_test[:10, 2:5])
print('\npredictions:')
print(reg.predict(X_test)[:10])
print('\ntargets:')
print(y_test[:10])

data:
[]

predictions:


NameError: name 'reg' is not defined